In [1]:
import pandas as pd
import numpy as np


In [2]:
from google.colab import files
uploaded = files.upload()  # Upload your Bigg_Boss_India CSV here


Saving Bigg_Boss_India (2).csv to Bigg_Boss_India (2).csv


In [3]:
df = pd.read_csv('Bigg_Boss_India (2).csv')


In [4]:
print(df.columns)


Index(['Language', 'Season Number', 'Name', 'Profession', 'Gender', 'Age',
       'Entry Date', 'Elimination Date', 'Elimination Week Number',
       'Contestant Number', 'Wild Card', 'Season Length',
       'Number of Housemates', 'Season Start Date', 'Season End Date',
       'Host Name', 'Guest Host Name', 'Prize Money (INR)', 'Broadcasted By',
       'OTT Platform', 'Average TRP', 'Most Viewed States', 'House Location',
       'OTT Season', 'Season Slogan', 'Season Theme',
       'Number of Evictions Faced', 'Number of Times Elected as Captain',
       'Number of Re-entries', 'Walked', 'Ejected', 'In Secret-Room',
       'Won Eviction Free Pass', 'Won Ticket to Finale',
       'Number of Times Best Performer', 'Number of Times Worst Performer',
       'Secret Task', 'Social Media Popularity', 'Finalist', 'Winner'],
      dtype='object')


In [5]:
# Create target variable
df['Winner_Flag'] = df['Winner'].fillna(0).astype(int)

# Features and target
X = df.select_dtypes(include=['int64', 'float64']).fillna(0)
y = df['Winner_Flag']

print("y defined, distribution:")
print(y.value_counts())




y defined, distribution:
Winner_Flag
0    1253
1      65
Name: count, dtype: int64


In [6]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42,
    stratify=y
)


In [7]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(
    n_estimators=200,
    random_state=42,
    class_weight='balanced'
)

rf.fit(X_train, y_train)
print("Model training completed")



Model training completed


In [8]:
from sklearn.metrics import accuracy_score, f1_score

y_pred = rf.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("F1 Score:", f1_score(y_test, y_pred))


Accuracy: 1.0
F1 Score: 1.0


In [9]:
# Predict probability instead of 0/1
df['Winner_Probability'] = rf.predict_proba(X)[:, 1]

# Check top probabilities
df[['Name', 'Season Number', 'Winner_Probability']] \
    .sort_values('Winner_Probability', ascending=False) \
    .head(10)


,Name,Season Number,Winner_Probability
1099,Vishal Nikam,3,1.0
1181,Sabumon Abdusamad,1,1.0
42,Vindu Dara Singh,3,1.0
1133,Suraj Chavan,5,1.0
600,Karthik Mahesh,10,1.0
369,Sana Makbul,OTT S3,1.0
1139,Aneek Dhar,1,1.0
591,Roopesh Shetty,9,1.0
945,Balaji Murugadoss,Ultimate,1.0
206,Dipika Kakar,12,1.0


In [10]:
predicted_winners = (
    df.sort_values('Winner_Probability', ascending=False)
      .groupby('Season Number')
      .head(1)
      [['Season Number', 'Name', 'Winner_Probability']]
)

print("Predicted Winners (Completed Seasons):")
predicted_winners


Predicted Winners (Completed Seasons):


,Season Number,Name,Winner_Probability
1099,3,Vishal Nikam,1.000
1181,1,Sabumon Abdusamad,1.000
1133,5,Suraj Chavan,1.000
600,10,Karthik Mahesh,1.000
369,OTT S3,Sana Makbul,1.000
591,9,Roopesh Shetty,1.000
945,Ultimate,Balaji Murugadoss,1.000
206,12,Dipika Kakar,1.000
1009,8,Muthukumaran Jegatheesan,0.995
472,4,Pratham,0.995


In [11]:
import numpy as np
import pandas as pd

future_df = pd.DataFrame({
    'Name': ['X', 'Y', 'Z'],
    'Season Number': ['20', '20', '20'],
    'Age': [28, 35, 37],
    'Elimination Week Number': [0, 0, 0],
    'Number of Times Elected as Captain': [1, 2, 3],
    'Number of Evictions Faced': [0, 1, 0],
    'Number of Re-entries': [0, 0, 0],
    'Number of Times Best Performer': [2, 3, 4],
    'Number of Times Worst Performer': [1, 0, 1],
    'Social Media Popularity': [85, 78, 92],
    'Contestant Number': [1, 2, 3]
})

future_df


,Name,Season Number,Age,Elimination Week Number,Number of Times Elected as Captain,Number of Evictions Faced,Number of Re-entries,Number of Times Best Performer,Number of Times Worst Performer,Social Media Popularity,Contestant Number
0,X,20,28,0,1,0,0,2,1,85,1
1,Y,20,35,0,2,1,0,3,0,78,2
2,Z,20,37,0,3,0,0,4,1,92,3


In [12]:
# Create aligned feature matrix
X_new = pd.DataFrame(0, index=future_df.index, columns=X_train.columns)

for col in X_train.columns:
    if col in future_df.columns:
        X_new[col] = future_df[col]

# Predict probability
future_df['Winner_Probability'] = rf.predict_proba(X_new)[:, 1]

# Top predicted winner
top_winner = future_df.sort_values('Winner_Probability', ascending=False).head(1)

print("Predicted Winner for Season 20:")
top_winner[['Name', 'Winner_Probability']]


Predicted Winner for Season 20:


,Name,Winner_Probability
0,X,0.03
